# Import libraries

In [ ]:
!pip install bitsandbytes>=0.39.0 accelerate>=0.20.0
!pip install transformers

In [ ]:
import inspect
import os
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from PIL import Image
from transformers import Blip2Processor, Blip2ForConditionalGeneration

# Instantiate model

In [ ]:
# Use BLIP-2 for the inference. via
# https://huggingface.co/Salesforce/blip2-flan-t5-xxl
processor = Blip2Processor.from_pretrained("Salesforce/blip2-flan-t5-xxl")
model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-flan-t5-xxl", device_map="auto", load_in_8bit=True)

# Mount drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Create directory to store inferences

In [ ]:
os.makedirs('/content/drive/MyDrive/stance_detection_datasets/inferences/predictions', exist_ok=True)

# Import datasets

In [ ]:
total_defense_memes = pd.read_csv('/content/drive/MyDrive/stance_detection_datasets/total_defense_memes/total_defense_memes.csv')
total_defense_memes = total_defense_memes[total_defense_memes['pillar'].str.contains('Others') == False]
total_defense_memes = total_defense_memes.reset_index(drop=True)

In [ ]:
total_defense_memes_captioned_Qwen_VL_Chat = pd.read_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/total_defense_memes_captioned_Qwen-VL-Chat.csv')
total_defense_memes_captioned_InternLM_XComposer = pd.read_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/total_defense_memes_captioned_InternLM-XComposer.csv')
total_defense_memes_captioned_llava = pd.read_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/total_defense_memes_captioned_llava-v1.5-13b.csv')
total_defense_memes_captioned_BLIP_2 = pd.read_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/total_defense_memes_captioned_BLIP-2.csv')

In [ ]:
total_defense_memes_captioned_Qwen_VL_Chat = pd.merge(total_defense_memes, total_defense_memes_captioned_Qwen_VL_Chat, how='left', on=['image'])
total_defense_memes_captioned_InternLM_XComposer = pd.merge(total_defense_memes, total_defense_memes_captioned_InternLM_XComposer, how='left', on=['image'])
total_defense_memes_captioned_llava = pd.merge(total_defense_memes, total_defense_memes_captioned_llava, how='left', on=['image'])
total_defense_memes_captioned_BLIP_2 = pd.merge(total_defense_memes, total_defense_memes_captioned_BLIP_2, how='left', on=['image'])

# Set prompts and define a function to call the model

In [ ]:
def prompt_with_caption(pillar, pillar_w_definition, caption):
    return inspect.cleandoc(f"""
    Description of this meme: \"\"\"
    {caption}
    \"\"\"
    {pillar_w_definition}
    Indicate whether this meme is against, neutral or supportive towards Singapore's {pillar}.
    Constraint: Without using any other words, answer either against, neutral, supportive.""")

In [ ]:
# Use BLIP-2 for the inference. via
# https://huggingface.co/Salesforce/blip2-flan-t5-xxl
def get_prediction(image, prompt):
    raw_image = Image.open(image).convert('RGB')
    inputs = processor(raw_image, prompt, return_tensors="pt").to("cuda")
    out = model.generate(**inputs, max_new_tokens=30)
    return processor.decode(out[0], skip_special_tokens=True)

# Define function to add definitions

In [ ]:
def definition(x):
    if x == 'Military Defence':
        return """Singapore's Military Defence: Strong and formidable defence force made up of Regulars and National Servicemen, and supported by the entire Singapore."""
    elif x == 'Civil Defence':
        return """Singapore's Civil Defence: Collective effort of the Singaporean society to spot signs of threats, respond effectively and recover quickly from crisis."""
    elif x == 'Economic Defence':
        return """Singapore's Economic Defence: Strong and resilient Singaporean economy that is globally competitive and able to bounce back from any crisis."""
    elif x == 'Social Defence':
        return """Singapore's Social Defence: Bonds that unite Singaporeans, built on trust and understanding among people of different races and religions, living in harmony and looking out for one another."""
    elif x == 'Psychological Defence':
        return """Singapore's Psychological Defence: The will and resolve to defend the Singaporean way of life and interests, the fighting spirit to overcome challenges together."""
    elif x == 'Digital Defence':
        return """Singapore's Digital Defence: Being secure, alert and responsible online."""

In [ ]:
total_defense_memes_captioned_Qwen_VL_Chat['pillar_w_definition'] = total_defense_memes_captioned_Qwen_VL_Chat['pillar'].apply(lambda x: definition(x))
total_defense_memes_captioned_InternLM_XComposer['pillar_w_definition'] = total_defense_memes_captioned_InternLM_XComposer['pillar'].apply(lambda x: definition(x))
total_defense_memes_captioned_llava['pillar_w_definition'] = total_defense_memes_captioned_llava['pillar'].apply(lambda x: definition(x))
total_defense_memes_captioned_BLIP_2['pillar_w_definition'] = total_defense_memes_captioned_BLIP_2['pillar'].apply(lambda x: definition(x))

# Define function to clean responses

In [ ]:
def remap(x):
    x = x.lower()
    if 'against' in x and 'neutral' not in x and 'supportive' not in x and 'not against' not in x:
        return 'Against'
    elif 'neutral' in x and 'against' not in x and 'supportive' not in x and 'not neutral' not in x:
        return 'Neutral'
    elif 'supportive' in x and 'neutral' not in x and 'against' not in x and 'not supportive' not in x:
        return 'Supportive'
    else:
        return None

# Call the `get_prediction` function and save inferences

GROUNDING [ABSENT] & PROMPT [CAPTION] & CAPTION [Qwen_VL_Chat]

In [ ]:
total_defense_memes_grounding_absent_prompt_with_caption_Qwen_VL_Chat = total_defense_memes_captioned_Qwen_VL_Chat.copy(deep=True)
total_defense_memes_grounding_absent_prompt_with_caption_Qwen_VL_Chat_images = total_defense_memes_grounding_absent_prompt_with_caption_Qwen_VL_Chat['image'].values
total_defense_memes_grounding_absent_prompt_with_caption_Qwen_VL_Chat_pillars = total_defense_memes_grounding_absent_prompt_with_caption_Qwen_VL_Chat['pillar'].values
total_defense_memes_grounding_absent_prompt_with_caption_Qwen_VL_Chat_definitions = total_defense_memes_grounding_absent_prompt_with_caption_Qwen_VL_Chat['pillar_w_definition'].values
total_defense_memes_grounding_absent_prompt_with_caption_Qwen_VL_Chat_captions = total_defense_memes_grounding_absent_prompt_with_caption_Qwen_VL_Chat['caption'].values
total_defense_memes_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'] = [get_prediction(image, prompt_with_caption(pillar, pillar_w_definition, caption)) for image, pillar, pillar_w_definition, caption in zip(total_defense_memes_grounding_absent_prompt_with_caption_Qwen_VL_Chat_images, total_defense_memes_grounding_absent_prompt_with_caption_Qwen_VL_Chat_pillars, total_defense_memes_grounding_absent_prompt_with_caption_Qwen_VL_Chat_definitions, total_defense_memes_grounding_absent_prompt_with_caption_Qwen_VL_Chat_captions)]
total_defense_memes_grounding_absent_prompt_with_caption_Qwen_VL_Chat.to_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/predictions/total_defense_memes_grounding[ABSENT]_caption[Qwen-VL-Chat]_prompt[CAPTION]_prediction[BLIP-2].csv', index=False)
total_defense_memes_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'] = total_defense_memes_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'].apply(lambda x: remap(x))
print(total_defense_memes_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'].value_counts())
print(total_defense_memes_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'].isna().sum())
total_defense_memes_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'] = total_defense_memes_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'].apply(lambda x: x if x is not None else np.random.choice(['Against', 'Neutral', 'Supportive']))
print(f1_score(total_defense_memes_grounding_absent_prompt_with_caption_Qwen_VL_Chat['stance'].values, total_defense_memes_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'].values, labels=['Against', 'Neutral', 'Supportive'], average='macro'))
print(classification_report(total_defense_memes_grounding_absent_prompt_with_caption_Qwen_VL_Chat['stance'].values, total_defense_memes_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'].values, labels=['Against', 'Neutral', 'Supportive']))

Neutral       635
Supportive    112
Against        24
Name: prediction, dtype: int64
0
0.3657501148138398
              precision    recall  f1-score   support

     Against       0.88      0.06      0.11       352
     Neutral       0.45      0.86      0.59       332
  Supportive       0.35      0.45      0.39        87

    accuracy                           0.45       771
   macro avg       0.56      0.46      0.37       771
weighted avg       0.63      0.45      0.35       771



GROUNDING [ABSENT] & PROMPT [CAPTION] & CAPTION [InternLM_XComposer]

In [ ]:
total_defense_memes_grounding_absent_prompt_with_caption_InternLM_XComposer = total_defense_memes_captioned_InternLM_XComposer.copy(deep=True)
total_defense_memes_grounding_absent_prompt_with_caption_InternLM_XComposer_images = total_defense_memes_grounding_absent_prompt_with_caption_InternLM_XComposer['image'].values
total_defense_memes_grounding_absent_prompt_with_caption_InternLM_XComposer_pillars = total_defense_memes_grounding_absent_prompt_with_caption_InternLM_XComposer['pillar'].values
total_defense_memes_grounding_absent_prompt_with_caption_InternLM_XComposer_definitions = total_defense_memes_grounding_absent_prompt_with_caption_InternLM_XComposer['pillar_w_definition'].values
total_defense_memes_grounding_absent_prompt_with_caption_InternLM_XComposer_captions = total_defense_memes_grounding_absent_prompt_with_caption_InternLM_XComposer['caption'].values
total_defense_memes_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'] = [get_prediction(image, prompt_with_caption(pillar, pillar_w_definition, caption)) for image, pillar, pillar_w_definition, caption in zip(total_defense_memes_grounding_absent_prompt_with_caption_InternLM_XComposer_images, total_defense_memes_grounding_absent_prompt_with_caption_InternLM_XComposer_pillars, total_defense_memes_grounding_absent_prompt_with_caption_InternLM_XComposer_definitions, total_defense_memes_grounding_absent_prompt_with_caption_InternLM_XComposer_captions)]
total_defense_memes_grounding_absent_prompt_with_caption_InternLM_XComposer.to_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/predictions/total_defense_memes_grounding[ABSENT]_caption[InternLM-XComposer]_prompt[CAPTION]_prediction[BLIP-2].csv', index=False)
total_defense_memes_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'] = total_defense_memes_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'].apply(lambda x: remap(x))
print(total_defense_memes_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'].value_counts())
print(total_defense_memes_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'].isna().sum())
total_defense_memes_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'] = total_defense_memes_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'].apply(lambda x: x if x is not None else np.random.choice(['Against', 'Neutral', 'Supportive']))
print(f1_score(total_defense_memes_grounding_absent_prompt_with_caption_InternLM_XComposer['stance'].values, total_defense_memes_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'].values, labels=['Against', 'Neutral', 'Supportive'], average='macro'))
print(classification_report(total_defense_memes_grounding_absent_prompt_with_caption_InternLM_XComposer['stance'].values, total_defense_memes_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'].values, labels=['Against', 'Neutral', 'Supportive']))

Token indices sequence length is longer than the specified maximum sequence length for this model (578 > 512). Running this sequence through the model will result in indexing errors


Neutral       637
Supportive    111
Against        23
Name: prediction, dtype: int64
0
0.32282928354754986
              precision    recall  f1-score   support

     Against       0.83      0.05      0.10       352
     Neutral       0.45      0.87      0.59       332
  Supportive       0.24      0.31      0.27        87

    accuracy                           0.43       771
   macro avg       0.51      0.41      0.32       771
weighted avg       0.60      0.43      0.33       771



GROUNDING [ABSENT] & PROMPT [CAPTION] & CAPTION [llava]

In [ ]:
total_defense_memes_grounding_absent_prompt_with_caption_llava = total_defense_memes_captioned_llava.copy(deep=True)
total_defense_memes_grounding_absent_prompt_with_caption_llava_images = total_defense_memes_grounding_absent_prompt_with_caption_llava['image'].values
total_defense_memes_grounding_absent_prompt_with_caption_llava_pillars = total_defense_memes_grounding_absent_prompt_with_caption_llava['pillar'].values
total_defense_memes_grounding_absent_prompt_with_caption_llava_definitions = total_defense_memes_grounding_absent_prompt_with_caption_llava['pillar_w_definition'].values
total_defense_memes_grounding_absent_prompt_with_caption_llava_captions = total_defense_memes_grounding_absent_prompt_with_caption_llava['caption'].values
total_defense_memes_grounding_absent_prompt_with_caption_llava['prediction'] = [get_prediction(image, prompt_with_caption(pillar, pillar_w_definition, caption)) for image, pillar, pillar_w_definition, caption in zip(total_defense_memes_grounding_absent_prompt_with_caption_llava_images, total_defense_memes_grounding_absent_prompt_with_caption_llava_pillars, total_defense_memes_grounding_absent_prompt_with_caption_llava_definitions, total_defense_memes_grounding_absent_prompt_with_caption_llava_captions)]
total_defense_memes_grounding_absent_prompt_with_caption_llava.to_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/predictions/total_defense_memes_grounding[ABSENT]_caption[llava-v1.5-13b]_prompt[CAPTION]_prediction[BLIP-2].csv', index=False)
total_defense_memes_grounding_absent_prompt_with_caption_llava['prediction'] = total_defense_memes_grounding_absent_prompt_with_caption_llava['prediction'].apply(lambda x: remap(x))
print(total_defense_memes_grounding_absent_prompt_with_caption_llava['prediction'].value_counts())
print(total_defense_memes_grounding_absent_prompt_with_caption_llava['prediction'].isna().sum())
total_defense_memes_grounding_absent_prompt_with_caption_llava['prediction'] = total_defense_memes_grounding_absent_prompt_with_caption_llava['prediction'].apply(lambda x: x if x is not None else np.random.choice(['Against', 'Neutral', 'Supportive']))
print(f1_score(total_defense_memes_grounding_absent_prompt_with_caption_llava['stance'].values, total_defense_memes_grounding_absent_prompt_with_caption_llava['prediction'].values, labels=['Against', 'Neutral', 'Supportive'], average='macro'))
print(classification_report(total_defense_memes_grounding_absent_prompt_with_caption_llava['stance'].values, total_defense_memes_grounding_absent_prompt_with_caption_llava['prediction'].values, labels=['Against', 'Neutral', 'Supportive']))

Neutral       639
Supportive    119
Against        13
Name: prediction, dtype: int64
0
0.32069891188118144
              precision    recall  f1-score   support

     Against       0.85      0.03      0.06       352
     Neutral       0.45      0.86      0.59       332
  Supportive       0.27      0.37      0.31        87

    accuracy                           0.43       771
   macro avg       0.52      0.42      0.32       771
weighted avg       0.61      0.43      0.32       771



GROUNDING [ABSENT] & PROMPT [CAPTION] & CAPTION [BLIP-2]

In [ ]:
total_defense_memes_grounding_absent_prompt_with_caption_BLIP_2 = total_defense_memes_captioned_BLIP_2.copy(deep=True)
total_defense_memes_grounding_absent_prompt_with_caption_BLIP_2_images = total_defense_memes_grounding_absent_prompt_with_caption_BLIP_2['image'].values
total_defense_memes_grounding_absent_prompt_with_caption_BLIP_2_pillars = total_defense_memes_grounding_absent_prompt_with_caption_BLIP_2['pillar'].values
total_defense_memes_grounding_absent_prompt_with_caption_BLIP_2_definitions = total_defense_memes_grounding_absent_prompt_with_caption_BLIP_2['pillar_w_definition'].values
total_defense_memes_grounding_absent_prompt_with_caption_BLIP_2_captions = total_defense_memes_grounding_absent_prompt_with_caption_BLIP_2['caption'].values
total_defense_memes_grounding_absent_prompt_with_caption_BLIP_2['prediction'] = [get_prediction(image, prompt_with_caption(pillar, pillar_w_definition, caption)) for image, pillar, pillar_w_definition, caption in zip(total_defense_memes_grounding_absent_prompt_with_caption_BLIP_2_images, total_defense_memes_grounding_absent_prompt_with_caption_BLIP_2_pillars, total_defense_memes_grounding_absent_prompt_with_caption_BLIP_2_definitions, total_defense_memes_grounding_absent_prompt_with_caption_BLIP_2_captions)]
total_defense_memes_grounding_absent_prompt_with_caption_BLIP_2.to_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/predictions/total_defense_memes_grounding[ABSENT]_caption[BLIP_2]_prompt[CAPTION]_prediction[BLIP-2].csv', index=False)
total_defense_memes_grounding_absent_prompt_with_caption_BLIP_2['prediction'] = total_defense_memes_grounding_absent_prompt_with_caption_BLIP_2['prediction'].apply(lambda x: remap(x))
print(total_defense_memes_grounding_absent_prompt_with_caption_BLIP_2['prediction'].value_counts())
print(total_defense_memes_grounding_absent_prompt_with_caption_BLIP_2['prediction'].isna().sum())
total_defense_memes_grounding_absent_prompt_with_caption_BLIP_2['prediction'] = total_defense_memes_grounding_absent_prompt_with_caption_BLIP_2['prediction'].apply(lambda x: x if x is not None else np.random.choice(['Against', 'Neutral', 'Supportive']))
print(f1_score(total_defense_memes_grounding_absent_prompt_with_caption_BLIP_2['stance'].values, total_defense_memes_grounding_absent_prompt_with_caption_BLIP_2['prediction'].values, labels=['Against', 'Neutral', 'Supportive'], average='macro'))
print(classification_report(total_defense_memes_grounding_absent_prompt_with_caption_BLIP_2['stance'].values, total_defense_memes_grounding_absent_prompt_with_caption_BLIP_2['prediction'].values, labels=['Against', 'Neutral', 'Supportive']))

Neutral       643
Supportive    119
Against         9
Name: prediction, dtype: int64
0
0.29640645906789037
              precision    recall  f1-score   support

     Against       0.56      0.01      0.03       352
     Neutral       0.43      0.84      0.57       332
  Supportive       0.25      0.34      0.29        87

    accuracy                           0.41       771
   macro avg       0.41      0.40      0.30       771
weighted avg       0.47      0.41      0.29       771

